## PR36 faster "accurate" position functional speed testing

This is run in the branch in effectively ska3-flight 2023.9rc1.

Result is that the "accurate" method is about 60% slower than the "fast" method. Given
the substantially better accuracy this level of slowdown is acceptable to have
"accurate" as the default.

In [1]:
import sys

sys.path.insert(0, "..")

import ska_sun
import astropy.units as u
from cxotime import CxoTime
import numpy as np

In [2]:
ska_sun.__file__

'/Users/aldcroft/git/ska_sun/validate/../ska_sun/__init__.py'

In [3]:
ska_sun.__version__

'3.12.1.dev6+g6abf649'

In [4]:
date0 = CxoTime("2019:001")
time0 = date0.secs
dates = [date0 + i_dt * u.day for i_dt in np.arange(0, 1000, 1)]
times = [date.secs for date in dates]

In [5]:
ska_sun.conf.sun_position_method_default

'accurate'

In [6]:
def run_func(func, times, **kwargs):
    """Run tests in a loop to guard against any caching"""
    for time in times:
        func(time=time, **kwargs)

In [7]:
%timeit run_func(ska_sun.position_accurate, times)

33.9 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit run_func(ska_sun.position, times)

40.4 ms ± 4.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit run_func(ska_sun.position_fast, times)

21.2 ms ± 258 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
with ska_sun.conf.set_temp("sun_position_method_default", "fast"):
    run_func(ska_sun.position, times)

24.7 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
